# Hidden Markov Models in python

In [15]:
import numpy as np

Initialising the parameter values. Fill in the values before running the code

In [4]:
# A is the transition probability matrix
# pi - initial state probability
# O - emission probability

A = np.array([[?, 0.1, 0.1],  # Row for H1
              [?, ?, 0.1],  # Row for H2
              [0.5, ?, 0.0]]) # Row for K2
pi = np.array([0.4, 0.35, 0.25])
O = np.array([[0.6, 0.2, 0.1,0.1], # Row for H1
              [?,?,?,?], # Row for H2
              [0.35, 0.2, 0.35,0.1]]) # Row for K2

- Hidden states are H1, H5, K2
- Observations are A1, A2, A3, A4. Here Ax stands for ATM x

In [5]:
states = H1, H5, K2 = 0, 1, 2
obs = A1,A2,A3,A4 = 0,1,2,3
observations = [A1,A1,A3,A1,A2,A3,A3]

## Exhaustive method

In [10]:
from itertools import product

def exhaustive(params, observations):
    pi, A, O = params
    M = len(observations)
    S = pi.shape[0]
    
    # track the running best sequence and its score
    best = (None, float('-inf'))
    # loop over the cartesian product of |states|^M
    for ss in product(range(S), repeat=M):
        # score the state sequence
        score = pi[ss[0]] * O[ss[0],observations[0]]
        for i in range(1, M):
            score *= A[ss[i-1], ss[i]] * O[ss[i], observations[i]]
        # update the running best
        if score > best[1]:
            best = (ss, score)  
    return best

In [12]:
exhaustive((pi, A, O), [A1,A1,A3,A1,A2,A3,A3])

((0, 0, 0, 0, 0, 1, 1), 1.4155776000000005e-05)

Yay, it got the same results as before. Note that the exhaustive method is practical on anything beyond toy data due to the nasty cartesian product. But it is worth doing to verify the Viterbi code above is getting the right results. 

# Viterbi Algorithm

In [13]:
def viterbi(params, observations):
    pi, A, O = params
    M = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf')
    backpointers = np.zeros((M, S), 'int')
    
    # base case
    alpha[0, :] = pi * O[:,observations[0]]
    
    # recursive case
    for t in range(1, M):
        for s2 in range(S):
            for s1 in range(S):
                score = alpha[t-1, s1] * A[s1, s2] * O[s2, observations[t]]
                if score > alpha[t, s2]:
                    alpha[t, s2] = score
                    backpointers[t, s2] = s1
    
    # now follow backpointers to resolve the state sequence
    ss = []
    ss.append(np.argmax(alpha[M-1,:]))
    for i in range(M-1, 0, -1):
        ss.append(backpointers[i, ss[-1]])
        
    return list(reversed(ss)), np.max(alpha[M-1,:])

In [16]:
res = viterbi((pi, A, O), [A1, A1, A3, A1, A2, A3, A3, A4, A4, A4, A4, A4, A4, A4, A1, A1, A3, A3, A4, A3, A4, A3, A4, A2, A2, A2, A4, A4, A3, A1, A2, A1, A1, A2, A3, A2, A2, A2, A4, A4, A4, A4, A3, A4, A3, A3, A2, A3])
print res

([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3.3294834175444772e-35)


Code Reference
- [COMP90042 Web Search and Text Analysis - Trevor Cohn, Univ. of Melbourne](http://people.eng.unimelb.edu.au/tcohn/comp90042.html)
- [NLP Course, Autumn 2016 Lecture Notes, IIT Kharagpur](https://github.com/krishnamrith12/NotebooksNLP)